In [25]:
import pandas as pd
import os
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /home/farkhad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


This code is used to open existing RAW article and filter it. We are removing stop words, and splitting text into list of words.

In [24]:
def articleClean():
    data = open("test_article.txt", "r")
    article = data.read()

    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(article)

    filtered_data = [] 
    for word in word_tokens:
        if word not in stop_words:
            filtered_data.append(word)
    
    string_set = ' '.join(str(e) for e in filtered_data)
    
    file = open("training_set.csv", "w+")
    file.write(string_set)
    file.close()
    
    

Executing articleClean() function.

In [36]:
articleClean()

Get all files as list

In [45]:
tech_dir_path = "/home/farkhad/jupy/articles/technology/training"
res = []

tech_res = os.listdir(tech_dir_path)
print(tech_res)

['training_tech26.txt', 'training_tech41.txt', 'training_tech68.txt', 'training_tech84.txt', 'training_tech52.txt', 'training_tech46.txt', 'training_tech35.txt', 'training_tech47.txt', 'training_tech74.txt', 'training_tech31.txt', 'training_tech9.txt', 'training_tech12.txt', 'training_tech53.txt', 'training_tech76.txt', 'training_tech79.txt', 'training_tech100.txt', 'training_tech91.txt', 'training_tech92.txt', 'training_tech85.txt', 'training_tech62.txt', 'training_tech5.txt', 'training_tech59.txt', 'training_tech57.txt', 'training_tech2.txt', 'training_tech63.txt', 'training_tech86.txt', 'training_tech40.txt', 'training_tech25.txt', 'training_tech61.txt', 'training_tech10.txt', 'training_tech99.txt', 'training_tech16.txt', 'training_tech13.txt', 'training_tech23.txt', 'training_tech93.txt', 'training_tech87.txt', 'training_tech22.txt', 'training_tech77.txt', 'training_tech73.txt', 'training_tech30.txt', 'training_tech83.txt', 'training_tech78.txt', 'training_tech38.txt', 'training_te

Get all sport files as list

In [46]:
sport_dir_path = "/home/farkhad/jupy/articles/sport/training"
res = []

sport_res = os.listdir(sport_dir_path)
print(sport_res)

['training_sport50.txt', 'training_sport25.txt', 'training_sport36.txt', 'training_sport78.txt', 'training_sport14.txt', 'training_sport13.txt', 'training_sport66.txt', 'training_sport47.txt', 'training_sport34.txt', 'training_sport54.txt', 'training_sport68.txt', 'training_sport7.txt', 'training_sport23.txt', 'training_sport93.txt', 'training_sport10.txt', 'training_sport53.txt', 'training_sport20.txt', 'training_sport11.txt', 'training_sport89.txt', 'training_sport95.txt', 'training_sport33.txt', 'training_sport85.txt', 'training_sport17.txt', 'training_sport52.txt', 'training_sport64.txt', 'training_sport4.txt', 'training_sport5.txt', 'training_sport82.txt', 'training_sport61.txt', 'training_sport69.txt', 'training_sport3.txt', 'training_sport31.txt', 'training_sport59.txt', 'training_sport75.txt', 'training_sport56.txt', 'training_sport90.txt', 'training_sport63.txt', 'training_sport6.txt', 'training_sport77.txt', 'training_sport74.txt', 'training_sport37.txt', 'training_sport72.tx

Now we need to arrange them in correct order.

In [ ]:
#Arrange code here